# Unsupervised training of the encoder layers

## Initial setup
### Imports

In [1]:
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import numpy as np
from datetime import datetime
from models import Autoencoder, EncoderStack
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import normalize
import dataset
import neptune
import neptune_tensorboard as neptune_tb
import os
import logging

2.2.0-dlenv


### Set experiment configuration

In [2]:
config = {
    "N_NODES": [2000, 1000, 500],
    "DROPOUT": [0.1],
    "BATCH_SIZE": 15,
    "EPOCHS": 150,
    "TEST_RATIO": 0.20,
    "DATA_BUCKET": "sdae-geo",
    "DATA_OBJECT": "GEO_data_batch_corr_final.csv",
    "DATA_LABELS": " GBM_class.csv",
    "VERBOSITY": 2,
    "LOG_DIR": "./log_dir",
    "PATIENCE":5
}


## Initialize Netptune and Tensorboard logging

In [3]:
os.environ['NEPTUNE_API_TOKEN']="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYjNiYmZhYjEtNzc3ZS00Y2NhLWI5NTgtYWU0MmQyMWJhM2I0In0="
os.environ['NEPTUNE_PROJECT']='AiPBBAND/SDAE'
os.environ['NEPTUNE_NOTEBOOK_ID']="ecd86e96-4da7-44e3-9a17-43da2dfcae35"
os.environ['NEPTUNE_NOTEBOOK_PATH']="constrained-SDAE/sdae.ipynb"

neptune.init(os.environ['NEPTUNE_PROJECT'], api_token=os.environ['NEPTUNE_API_TOKEN'])

logger = logging.getLogger("SDAE")
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

experiment = neptune.create_experiment(name='configuration', params=config, 
    logger=logger, upload_source_files=["*.py"])

os.environ['EXP_DIR'] = os.path.join(config["LOG_DIR"], experiment.id)
os.mkdir(os.environ['EXP_DIR'])

logger.info("project directory: {}".format(os.environ['EXP_DIR']))
!neptune tensorboard ${EXP_DIR} --project ${NEPTUNE_PROJECT}
%load_ext tensorboard

NVMLError: NVML Shared Library Not Found - GPU usage metrics may not be reported.


https://ui.neptune.ai/AiPBBAND/SDAE/e/SAN-5


project directory: ./log_dir/SAN-5


### Start tensorboard server
Tensorboard by running the following command in a terminal:

In [4]:
print("tensorboard --logdir {} --bind_all".format(os.environ['EXP_DIR']))

tensorboard --logdir ./log_dir/SAN-5 --bind_all


**Tensorboard cannot server over HTTPS, use external HTTP url: http://34.77.45.86:6006/**

# Load and preprocess the data

## Load data from Google Storage

In [5]:
dataframe = dataset.load_gs_data(config['DATA_BUCKET'], config['DATA_OBJECT'], os.environ['EXP_DIR'])

Initializing dataset...
File GEO_data_batch_corr_final.csv downloaded to ./log_dir/SAN-5/GEO_data_batch_corr_final.csv.
Loaded 954 samples with 6785 features.


## Normalize data feature-wize

In [6]:
data = dataframe.values
data = normalize(data)

## Split into training and testing sets

In [7]:
rs = ShuffleSplit(n_splits=1, test_size=config['TEST_RATIO'], random_state=0)
split_itterator = rs.split(data)
i_train, i_test = next(split_itterator)
train_path = os.path.join(os.environ['EXP_DIR'], "train_indices.npy")
test_path = os.path.join(os.environ['EXP_DIR'], "test_indices.npy")
np.save(train_path, i_train)
np.save(test_path, i_test)
neptune.log_artifact(train_path)
neptune.log_artifact(test_path)

x_train, x_test = data[i_train], data[i_test]

logger.info("{}% of samples for training: {} training, {} testing.".format(int(config['TEST_RATIO']*100), len(i_train), len(i_test)))

20% of samples for training: 763 training, 191 testing.


# Train encoder layers

In [8]:
logger.info("Starting training for {} encoder layers.".format(len(config["N_NODES"])))

x_train_out, x_test_out = x_train, x_test
for idx, num_hidden in enumerate(config["N_NODES"]):
    logger.info("Training layer {} with {} hidden nodes..".format(idx, num_hidden))
    encoder = Autoencoder(x_train_out.shape[1], num_hidden, os.environ['EXP_DIR'])
    
    recon_mse = encoder.fit(x_train_out, x_test_out, batch_size=config["BATCH_SIZE"], 
        num_epochs=config["EPOCHS"], verbose=config["VERBOSITY"], patience=config["PATIENCE"])
    
    x_train_out = encoder.encoder_model.predict(x_train_out)
    x_test_out = encoder.encoder_model.predict(x_test_out)
    logger.info("Training losss for layer {}: {} ".format(idx, recon_mse[0]))
    logger.info("Testing loss for layer {}: {} ".format(idx, recon_mse[1]))
    
    model_path = os.path.join(os.environ['EXP_DIR'], "encoders", "model_{}_{}".format(idx,num_hidden))
    encoder.encoder_model.save(model_path)
    neptune.log_artifact(model_path)
    neptune.set_property('encoder{}'.format(idx), model_path)




Starting training for 3 encoder layers.
Training layer 0 with 2000 hidden nodes..


Epoch 1/150
51/51 [==============================] - 12s 228ms/step - loss: 0.1745 - val_loss: 0.0683
Epoch 2/150
51/51 [==============================] - 11s 219ms/step - loss: 0.0202 - val_loss: 0.0055
Epoch 3/150
51/51 [==============================] - 11s 220ms/step - loss: 0.0029 - val_loss: 0.0035
Epoch 4/150
51/51 [==============================] - 11s 216ms/step - loss: 0.0011 - val_loss: 6.8450e-04
Epoch 5/150
51/51 [==============================] - 11s 225ms/step - loss: 4.6739e-04 - val_loss: 3.5521e-04
Epoch 6/150
51/51 [==============================] - 11s 218ms/step - loss: 2.4919e-04 - val_loss: 2.0065e-04
Epoch 7/150
51/51 [==============================] - 11s 219ms/step - loss: 1.4786e-04 - val_loss: 1.1501e-04
Epoch 8/150
51/51 [==============================] - 11s 217ms/step - loss: 9.2820e-05 - val_loss: 7.1589e-05
Epoch 9/150
51/51 [==============================] - 11s 212ms/step - loss: 6.4068e-05 - val_loss: 4.7529e-05
Epoch 10/150
51/51 [==================

Trained model saved at: ./log_dir/SAN-5/autoencoder-2000
Training losss for layer 0: 4.5784613575960975e-06 
Testing loss for layer 0: 4.584167527355021e-06 


INFO:tensorflow:Assets written to: ./log_dir/SAN-5/encoders/model_0_2000/assets


Training layer 1 with 1000 hidden nodes..


...

INFO:tensorflow:Assets written to: ./log_dir/SAN-5/encoders/model_1_1000/assets


Training layer 2 with 500 hidden nodes..


Epoch 1/150
51/51 [==============================] - 1s 25ms/step - loss: 0.0174 - val_loss: 0.0130
Epoch 2/150
51/51 [==============================] - 1s 22ms/step - loss: 0.0100 - val_loss: 0.0075
Epoch 3/150
51/51 [==============================] - 1s 22ms/step - loss: 0.0069 - val_loss: 0.0060
Epoch 4/150
51/51 [==============================] - 1s 22ms/step - loss: 0.0048 - val_loss: 0.0039
Epoch 5/150
51/51 [==============================] - 1s 21ms/step - loss: 0.0034 - val_loss: 0.0027
Epoch 6/150
51/51 [==============================] - 1s 21ms/step - loss: 0.0024 - val_loss: 0.0020
Epoch 7/150
51/51 [==============================] - 1s 22ms/step - loss: 0.0017 - val_loss: 0.0013
Epoch 8/150
51/51 [==============================] - 1s 21ms/step - loss: 0.0012 - val_loss: 8.8582e-04
Epoch 9/150
51/51 [==============================] - 1s 19ms/step - loss: 9.1090e-04 - val_loss: 8.2162e-04
Epoch 10/150
51/51 [==============================] - 1s 20ms/step - loss: 7.0669e-04 - 

Trained model saved at: ./log_dir/SAN-5/autoencoder-500
Training losss for layer 2: 7.018666565272724e-06 
Testing loss for layer 2: 7.080084287736099e-06 


INFO:tensorflow:Assets written to: ./log_dir/SAN-5/encoders/model_2_500/assets


In [9]:
neptune.stop()